In [ ]:
# Imports

import os
import json
from typing import Dict, List
import openai
from tqdm import tqdm
from pydantic import BaseModel
from dotenv import load_dotenv
from collections import Counter
import random

load_dotenv()
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [49]:
# JSON schema for gpt output (structured outputs)

class Speech(BaseModel):
    speech_id: str
    topics: List[str]
    
class Speeches(BaseModel):
    speeches: List[Speech]

In [50]:
def read_speech_file(file_path: str) -> Dict[str, str]:
    """
    Reads a speech file and returns a dictionary mapping speech_ids to speeches.
    Only includes speeches with more than 35 words and less than 400 words.
    Handles and logs problematic lines without stopping the process.

    Args:
        file_path (str): Path to the speech file

    Returns:
        Dict[str, str]: Dictionary mapping speech_ids to speeches
    """
    speeches = {}
    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
        # Skip header line
        next(file)
        for line_number, line in enumerate(file, start=2):  # Start at 2 to account for header
            try:
                # Replace unknown characters with space
                cleaned_line = line.replace('�', ' ')
                # Split on pipe character - column delimmiter
                parts = line.strip().split('|')
                if len(parts) == 2:
                    speech_id, speech = parts
                    # Only add speech if it has more than 35 words and less than 400 words
                    word_count = len(speech.split())
                    if 35 < word_count < 400:
                        speeches[speech_id] = speech
            except Exception as e:
                # Log the problematic line and continue
                print(f"Error processing line {line_number}: {line.strip()}\nException: {e}")
    return speeches

In [51]:
def chunk_speeches(speeches: Dict[str, str], max_chunk_size: int = 20000) -> List[Dict[str, str]]:
    """
    Splits speeches into chunks while keeping individual speeches intact.

    Args:
        speeches (Dict[str, str]): Dictionary mapping speech_ids to speeches
        max_chunk_size (int): Maximum size of each chunk in characters

    Returns:
        List[Dict[str, str]]: List of dictionaries mapping speech_ids to speechess
            such that the total size of speeches in each dictionary is less than or
            equal to max_chunk_size
    """
    chunks = []
    current_chunk = {}
    current_size = 0
    
    for speech_id, speech in speeches.items():
        speech_size = len(speech)
        
        # If adding this speech would exceed max size and we already have speeches,
        # start a new chunk
        if current_size + speech_size > max_chunk_size and current_chunk:
            chunks.append(current_chunk)
            current_chunk = {}
            current_size = 0
            
        current_chunk[speech_id] = speech
        current_size += speech_size
    
    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

In [52]:
def extract_speech_topics(speeches: Dict[str, str]):
    """
    Uses OpenAI's API to extract topics from the given speeches.

    Args:
        speeches (Dict[str, str]): Dictionary mapping speech_ids to speeches

    Returns:
        JSON Schema formatted output. Key is "speeches" and value is a list of objects
            where each object has a "speech_id" and a "topics" list.
    """
    # Prepare the speeches for analysis
    speeches_text = "\n\n".join([f"Speech ID: {id}\nContent: {text}" for id, text in speeches.items()])
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": """
                    Analyze each speech and identify its relevant topics. Topics should be chosen from this fixed list:
                    - Governance and Democracy
                    - Economy and Jobs
                    - Health and Social Services
                    - Education and Innovation
                    - Environment and Energy
                    - Defense and Security
                    - Immigration and Border Policy
                    - Justice and Civil Rights
                    - Infrastructure and Transportation
                    - Budget and Fiscal Responsibility

                    For each speech, assign one or more topics that best match its content.
                    Output should be in JSON format containing a list of objects, each with a speech_id and its corresponding topics list.
                    Be precise and thorough in topic assignment. Choose ONLY from the provided list of topics.
                    """
                },
                {
                    "role": "user",
                    "content": speeches_text
                }
            ],
            response_format={
                "type": "json_schema",
                "json_schema": 
                    {
                        "name":"_", 
                        "schema": Speeches.model_json_schema()
                    }
            },
            temperature=0.3,
            timeout=600  # 10 minute timeout
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during API call: {str(e)}")
        raise

In [53]:
def process_speech_file(file_path: str, output_dir: str, max_chunks: int=None):
    """
    Process a speech file and save the results.

    Args:
        file_path (str): Path to the speech file
        output_dir (str): Directory to save the results
        max_chunks (int): Maximum number of chunks to process

    Returns:
        None
    """
    print(f"Processing file: {file_path}")
    
    # Read speeches - > turn into dictionary of speech_id: speech_text
    speeches = read_speech_file(file_path)
    print(f"Found {len(speeches)} speeches")
    
    # Split into chunks
    chunks = chunk_speeches(speeches)
    print(f"Split into {len(chunks)} chunks")
    
    # Process each chunk and combine results
    all_results = {}

    # don't process chunks sequentially, instead loop through a
    # random order of indices to process the chunks in a random order
    chunk_indices = list(range(len(chunks)))
    random.shuffle(chunk_indices)
    chunk_count = 0
    for i in tqdm(chunk_indices, desc="Processing chunks"):
        chunk_count += 1
        chunk = chunks[i]
        try:
            # Get topics for speeches in this chunk
            topics_json = extract_speech_topics(chunk)
            topics_results = json.loads(topics_json)
            
            # Combine speech text with topics
            for speech in topics_results["speeches"]:
                speech_id = speech["speech_id"]
                all_results[speech_id] = {
                    "speech": speeches[speech_id],
                    "topics": speech["topics"]
                }
            
            # print for every 10 chunks we process
            if (chunk_count) % 10 == 0:
                print(f"Successfully processed chunk {chunk_count}")

            # Stop if we've reached the maximum number of chunks (if we set a limit)
            if max_chunks and chunk_count >= max_chunks:
                break
        except Exception as e:
            print(f"Error processing chunk {i}: {str(e)}")
            continue

    # Save results
    output_file = os.path.join(output_dir, os.path.basename(file_path).replace('.txt', '_gpt_topic_labels.json'))
    os.makedirs(output_dir, exist_ok=True)
    
    with open(output_file, 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\nResults saved to {output_file}")

In [ ]:
def analyze_topic_frequencies(input_dir: str, suffix:str='_gpt_topic_labels.json') -> Dict[str, int]:
    """
    Analyze JSON files in the input directory and count topic frequencies.
    
    Args:
        input_dir: Directory containing the speech analysis JSON files
        suffix: Suffix of the JSON files to analyze
        
    Returns:
        Dictionary mapping topics to their frequencies
    """
    # Initialize counter for topics
    topic_counter = Counter()
    
    # Get all JSON files in directory
    json_files = [f for f in os.listdir(input_dir) if f.endswith(suffix)]
    
    # Process each file
    for filename in tqdm(json_files, desc="Processing files"):
        file_path = os.path.join(input_dir, filename)
        
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # Go through each speech
            for speech_id, speech_data in data.items():
                # Add each topic to our counter
                topic_counter.update(speech_data['topics'])
    
    # Convert Counter to dictionary and sort by frequency
    topic_frequencies = dict(sorted(topic_counter.items(), key=lambda x: x[1], reverse=True))
    
    return topic_frequencies

In [55]:
def print_topic_statistics(topic_frequencies: Dict[str, int]):
    """
    Print formatted statistics about topic frequencies.

    Args:
        topic_frequencies: Dictionary mapping topics to their frequencie
    
    Returns:
        None
    """
    print("\nTopic Frequencies:")
    print("-" * 40)
    
    # Find the longest topic name for formatting
    max_topic_length = max(len(topic) for topic in topic_frequencies.keys())
    
    # Print each topic and its count
    for topic, count in topic_frequencies.items():
        print(f"{topic:<{max_topic_length}} : {count:>6}")
    
    print("-" * 40)
    print(f"Total topics mentioned: {sum(topic_frequencies.values())}")

In [ ]:
def process_speech_files(speech_files: List[str], input_dir: str, output_dir: str, max_chunks: int=None):
    """
    Process multiple speech files.

    Args:
        speech_files: List of speech file names
        input_dir: Directory containing the speech files
        output_dir: Directory to save the results
        max_chunks: Maximum number of chunks to process for each file

    Returns:
        None
    """
    for file_name in tqdm(speech_files, desc="Processing files"):
        file_path = os.path.join(input_dir, file_name)
        # get the specific file number, like for example 80 from 'speeches_080.txt'
        file_num = file_name.split('_')[1].split('.')[0]
        process_speech_file(file_path, output_dir, max_chunks)
        print(f"topic frequency graph for {file_num+'_gpt_topic_labels.json'}:")
        # print topic frequencies for each file as we process them
        topic_frequencies_interim = analyze_topic_frequencies(output_dir, suffix=file_num+'_gpt_topic_labels.json')
        print_topic_statistics(topic_frequencies_interim)

In [ ]:
if __name__ == "__main__":
    # first, process the hein-bound files (they only go up to 111)
    input_dir = "../hein-bound"
    # save outputs to 'outputs' directory
    output_dir = "outputs"
    speech_files = [f for f in os.listdir(input_dir) 
                    if f.startswith("speeches_") and f.endswith(".txt") 
                    and f[9:-4].isdigit() and 79 <= int(f[9:-4]) <= 111]
    # sort the speech files
    speech_files.sort()
    process_speech_files(speech_files, input_dir, output_dir, max_chunks=100)

    # then, process the hein-daily files (they go from 112 to 114, the congresses not covered by hein-bound)
    input_dir = "../hein-daily"
    speech_files = [f for f in os.listdir(input_dir) 
                    if f.startswith("speeches_") and f.endswith(".txt") 
                    and f[9:-4].isdigit() and 112 <= int(f[9:-4]) <= 114]
    process_speech_files(speech_files, input_dir, output_dir, max_chunks=100)

    # Analyze topic frequencies for all files at the very end
    input_dir_freq = "outputs"
    topic_frequencies = analyze_topic_frequencies(input_dir_freq)
    print_topic_statistics(topic_frequencies)

Processing files:   0%|          | 0/33 [00:00<?, ?it/s]

Processing file: ../hein-bound/speeches_079.txt
Found 77145 speeches
Split into 2493 chunks


Successfully processed chunk 10


Successfully processed chunk 20


Successfully processed chunk 30


...

Successfully processed chunk 100

Results saved to outputs/speeches_112_gpt_topic_labels.json
topic frequency graph for 112_gpt_topic_labels.json


Processing files: 100%|██████████| 3/3 [20:47<00:00, 415.79s/it]



Topic Frequencies:
----------------------------------------
Economy and Jobs                  :    476
Budget and Fiscal Responsibility  :    414
Health and Social Services        :    288
Justice and Civil Rights          :    285
Defense and Security              :    260
Governance and Democracy          :    234
Education and Innovation          :    210
Environment and Energy            :    110
Infrastructure and Transportation :    105
Immigration and Border Policy     :     54
Agriculture                       :      3
Trade and Foreign Relations       :      3
Community Service                 :      3
Water Issues                      :      1
Tourism                           :      1
Culture and Society               :      1
----------------------------------------
Total topics mentioned: 2448


Processing files: 100%|██████████| 38/38 [00:00<00:00, 150.34it/s]


Topic Frequencies:
----------------------------------------
Budget and Fiscal Responsibility     :  15578
Justice and Civil Rights             :  15305
Economy and Jobs                     :  15189
Governance and Democracy             :  15149
Defense and Security                 :  12459
Health and Social Services           :   9493
Education and Innovation             :   5703
Environment and Energy               :   4288
Infrastructure and Transportation    :   3924
Immigration and Border Policy        :   1656
Agriculture                          :    249
Housing and Urban Development        :    198
Housing and Social Services          :     63
Energy and Environment               :     36
Housing and Community Development    :     26
Labor Relations                      :     25
Foreign Policy                       :     22
Culture and Community                :     20
Culture and Arts                     :     20
Trade                                :     17
Trade Agreements   